# Road Test Sequential Design

In [1]:
import ema_workbench
ema_workbench.__version__

/Users/jpn/anaconda/envs/EMAT/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/Users/jpn/anaconda/envs/EMAT/lib/python3.6/site-packages/distributed/utils.py:136: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [Errno 51] Network is unreachable
  RuntimeWarning,


'2.1.CS1'

In [2]:
import emat
emat.__version__

'0.1.3'

In [3]:
import os, numpy, pandas, functools
from emat.util.xmle import Show

In [4]:
logger = emat.util.loggers.log_to_stderr(20, True)

In [5]:
road_test_scope_file = emat.package_file('model','tests','road_test.yaml')

In [6]:
road_scope = emat.Scope(road_test_scope_file)
road_scope

<emat.Scope with 2 constants, 7 uncertainties, 4 levers, 7 measures>

In [7]:
road_scope.info()

name: EMAT Road Test
desc: prototype run
constants:
  free_flow_time = 60
  initial_capacity = 100
uncertainties:
  alpha = 0.1 to 0.2
  beta = 3.5 to 5.5
  input_flow = 80 to 150
  value_of_time = 0.001 to 0.08
  unit_cost_expansion = 95.0 to 145.0
  interest_rate = 0.025 to 0.04
  yield_curve = -0.0025 to 0.02
levers:
  expand_capacity = 0.0 to 100.0
  amortization_period = 15 to 50
  debt_type = categorical
  interest_rate_lock = boolean
measures:
  no_build_travel_time
  build_travel_time
  time_savings
  value_of_time_savings
  net_benefits
  cost_of_capacity_expansion
  present_cost_expansion


In [8]:
emat_db = emat.SQLiteDB()

[00:08.58] MainProcess/INFO: running script scope.sql
[00:08.59] MainProcess/INFO: running script exp_design.sql
[00:08.59] MainProcess/INFO: running script meta_model.sql


In [9]:
road_scope.store_scope(emat_db)

In [10]:
from emat.experiment.experimental_design import design_experiments

In [11]:
design = design_experiments(road_scope, db=emat_db, n_samples_per_factor=10, sampler='lhs')
design.head()

,alpha,amortization_period,beta,debt_type,expand_capacity,input_flow,interest_rate,interest_rate_lock,unit_cost_expansion,value_of_time,yield_curve,free_flow_time,initial_capacity
experiment,,,,,,,,,,,,,
1,0.184682,38,5.237143,Rev Bond,18.224793,115,0.031645,False,118.213466,0.022518,0.015659,60,100
2,0.166133,36,4.121963,Paygo,87.525790,129,0.037612,True,141.322696,0.052306,0.007307,60,100
3,0.198937,44,4.719838,GO Bond,45.698048,105,0.028445,False,97.783320,0.011923,-0.001545,60,100
4,0.158758,42,4.915816,GO Bond,51.297546,113,0.036234,True,127.224901,0.077527,0.004342,60,100
5,0.157671,42,3.845952,Paygo,22.824149,133,0.039257,False,107.820482,0.027270,0.001558,60,100


## Core Model

In [12]:
from emat.model.core_python import PythonCoreModel
from emat.model.core_python import Road_Capacity_Investment

In [13]:
m = PythonCoreModel(Road_Capacity_Investment, scope=road_scope, db=emat_db)

In [14]:
lhs_results = m.run_experiments(design_name='lhs')
lhs_results.head()

[00:08.79] MainProcess/INFO: performing 110 scenarios/policies * 1 model(s) = 110 experiments
[00:08.82] MainProcess/INFO: performing experiments sequentially
[00:08.83] MainProcess/INFO: 11 cases completed
[00:08.83] MainProcess/INFO: 22 cases completed
[00:08.84] MainProcess/INFO: 33 cases completed
[00:08.85] MainProcess/INFO: 44 cases completed
[00:08.85] MainProcess/INFO: 55 cases completed
[00:08.86] MainProcess/INFO: 66 cases completed
[00:08.87] MainProcess/INFO: 77 cases completed
[00:08.88] MainProcess/INFO: 88 cases completed
[00:08.88] MainProcess/INFO: 99 cases completed
[00:08.89] MainProcess/INFO: 110 cases completed
[00:08.89] MainProcess/INFO: experiments finished


,alpha,beta,input_flow,value_of_time,unit_cost_expansion,interest_rate,yield_curve,expand_capacity,amortization_period,debt_type,interest_rate_lock,no_build_travel_time,build_travel_time,time_savings,value_of_time_savings,net_benefits,cost_of_capacity_expansion,present_cost_expansion
experiment,,,,,,,,,,,,,,,,,,
1,0.184682,5.237143,115,0.022518,118.213466,0.031645,0.015659,18.224793,38,Rev Bond,False,83.038716,69.586789,13.451927,34.835362,-79.515515,114.350877,2154.415985
2,0.166133,4.121963,129,0.052306,141.322696,0.037612,0.007307,87.525790,36,Paygo,True,88.474313,62.132583,26.341730,177.741192,-205.321480,383.062672,12369.380535
3,0.198937,4.719838,105,0.011923,97.783320,0.028445,-0.001545,45.698048,44,GO Bond,False,75.027180,62.543328,12.483852,15.629037,-151.944318,167.573355,4468.506839
4,0.158758,4.915816,113,0.077527,127.224901,0.036234,0.004342,51.297546,42,GO Bond,True,77.370428,62.268768,15.101660,132.298475,-167.624871,299.923347,6526.325171
5,0.157671,3.845952,133,0.027270,107.820482,0.039257,0.001558,22.824149,42,Paygo,False,88.328990,72.848428,15.480561,56.146908,-3.972940,60.119848,2460.910705


In [15]:
mm = m.create_metamodel_from_design('lhs')
mm

<emat.PythonCoreModel "EMATMeta", metamodel_id=1 with 2 constants, 7 uncertainties, 4 levers, 7 measures>

In [16]:
mm_cv = mm.function.cross_val_scores()

In [17]:
mm_cv

no_build_travel_time          0.975653
build_travel_time             0.984832
time_savings                  0.880716
value_of_time_savings         0.807826
net_benefits                  0.825774
cost_of_capacity_expansion    0.834869
present_cost_expansion        0.932757
dtype: float64

In [18]:
#mm_cv_ = mm.function.cross_val_scores_()

In [19]:
# def cross_val_scores_(self, cv=5, gpr_only=False):
#     """
#     Calculate the cross validation scores for this meta-model.

#     Args:
#         cv (int, default 5): The number of folds to use in
#             cross-validation.
#         gpr_only (bool, default False): Whether to limit the
#             cross-validation analysis to only the GPR step (i.e.,
#             to measure the improvement in meta-model fit from
#             using the GPR-based meta-model, over and above
#             using the linear regression meta-model alone.)

#     Returns:
#         pandas.Series: The cross-validation scores, by output.

#     """
#     if self.sample_stratification is not None:
#         from ..multitarget.splits import ExogenouslyStratifiedKFold
#         cv = ExogenouslyStratifiedKFold(exo_data=self.sample_stratification, n_splits=cv)

#     if gpr_only:
#         residuals = self.regression.residual_predict(self.input_sample)
#         regression = multitarget.MultipleTargetRegression()
#         return regression.cross_val_scores(self.input_sample, residuals, cv=cv)
#     return self.regression.cross_val_scores_(self.input_sample, self.output_sample, cv=cv)


In [20]:
# mm.function.regression.score(mm.function.input_sample, mm.function.output_sample)

In [21]:
# mm.function.regression.scores(mm.function.input_sample, mm.function.output_sample)

In [22]:
# mm.function.regression

In [23]:
mm_cv_ = cross_val_scores_(mm.function, cv=5)
mm_cv_

no_build_travel_time          0.993385
build_travel_time             0.984056
time_savings                  0.882865
value_of_time_savings         0.868693
net_benefits                  0.755810
cost_of_capacity_expansion    0.897744
present_cost_expansion        0.931553
dtype: float64

In [24]:
cross_val_scores_(mm.function, cv=5)

no_build_travel_time          0.996059
build_travel_time             0.979534
time_savings                  0.924309
value_of_time_savings         0.806513
net_benefits                  0.763217
cost_of_capacity_expansion    0.836336
present_cost_expansion        0.944906
dtype: float64

### Loeppky Augmentation

In [ ]:
n_augmentations = 3
aug_batch_size = 10
aug_design = {}
aug_results = {}
aug_mm = {-1: mm}
aug_cv = {-1: mm_cv}
aug_names = ['lhs',]

In [ ]:
for aug in range(n_augmentations):
    
    design_pool = design_experiments(
        road_scope, 
        n_samples=5000, 
        sampler='ulhs', 
        random_seed=42+aug,
    )
    
    aug_name = f'la_{aug}'
    aug_design[aug] = aug_mm[aug-1].function.pick_new_experiments(
            design_pool,
            batch_size=aug_batch_size,
            output_focus=dict(1-aug_cv[aug-1]),
            scope=road_scope,
            db=emat_db,
            design_name=aug_name,
    )
    aug_results[aug] = m.run_experiments(design_name=aug_name)
    aug_names.append(aug_name)
    aug_mm[aug] = m.create_metamodel_from_designs(aug_names)
    aug_cv[aug] = aug_mm[aug].function.cross_val_scores()
    print(f"-- Cross Validation Loeppky {aug} --\n", aug_cv[aug])
        
#     j = aug_mm[aug].run_experiments(grid_df[['x1','x2']], db=False)
#     j_stderr = aug_mm[aug].function.compute_std(grid_df[['x1','x2']])
    
#     for cc in [1,2,3,4]:
#         fig, axs = plt.subplots(1,4, figsize=(13,3))
#         axs[0].scatter(grid_df.x1, grid_df.x2, c=grid_df[f'c{cc}'])
#         axs[0].set_title(f'Core{cc}')
#         axs[1].scatter(j.x1, j.x2, c=j[f'c{cc}'])
#         axs[1].set_title(f'Loeppky MetaModel{cc}')
#         diffs = grid_df[f'c{cc}']-j[f'c{cc}']
#         maxdiff = numpy.fabs(diffs).max()
#         axs[2].scatter(j.x1, j.x2, c=diffs, cmap='PiYG', vmin=-maxdiff, vmax=maxdiff)
#         axs[2].set_title(f'Error in MetaModel{cc}')

#         axs[3].scatter(j.x1, j.x2, c=j_stderr[f'c{cc}'])
#         axs[3].set_title(f'Std Err of MetaModel{cc}')

#         plt.show()



In [ ]:
df = emat_db.read_experiment_all(scope_name=emat_db.read_scope_names()[0], design=None)

In [ ]:
(df.expand_capacity * df.unit_cost_expansion)

In [ ]:
df_p = emat_db.read_experiment_parameters(scope_name=emat_db.read_scope_names()[0], design=None)

In [ ]:
pandas.concat([
    df.present_cost_expansion.rename("CoreModel"),
    mm.function.predict(df_p, trend_only=True)['present_cost_expansion'].rename('Trend')
], axis=1)

In [ ]:
mm.function.predict(df_p, trend_only=True)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:

fig, ax = plt.subplots()
ax.scatter(
    df.unit_cost_expansion, 
    df.expand_capacity, 
    c=df.present_cost_expansion,
)
plt.show()
plt.close(fig)


In [ ]:
df.info()

In [ ]:
aug_mm[0].function.cross_val_scores()

In [ ]:
aug_mm[2].function.input_sample

In [ ]:
# aug_cv[aug] = aug_mm[aug].function.cross_val_scores()

